__Завгородняя Марина Игоревна Группа MADE-DS-22__ 

Второе домашнее задание по курсу "Продвинутое машинное обучение":

- MCMC-сэмплирование и «пляшущие человечки»

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter

import random
import re

In [ ]:
from collections import Counter

from sklearn.base import BaseEstimator, TransformerMixin


class TextEncryptor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.corpora = None
        self.freq_dict = None
        self.random_mapping = None

    def fit(self, corpora):
        """

        :param corpora: частотный словарь корпуса известного языка
        """
        self.corpora = corpora
        self.freq_dict = self._get_freq_dict(self.corpora)
        self.random_mapping = self._create_random_mapping(self.freq_dict)

    def decrypt_frequency_method(self, unknown_text: str):
        """Расшифровывает текст с помощью частотного метода.
        unknown_text: текст для расшифровки"""
        freq_dict_unknown_text = self._get_freq_dict(unknown_text)
        mapping_for_decrypt = self._create_mapping_by_freq_dicts(
            self.freq_dict, freq_dict_unknown_text
        )
        return self._map_text(unknown_text, mapping_for_decrypt)

    def encrypt_random(self, text: str):
        return self._map_text(text, self.random_mapping)

    @staticmethod
    def _get_freq_dict(text) -> dict:
        """ Возвращает dict частотности символов {'x': n} в тексте text """
        counter = Counter(text)
        sorted_dict = {
            k: v for k, v in sorted(dict(counter).items(), key=lambda item: -item[1])
        }
        return sorted_dict

    @staticmethod
    def _create_random_mapping(freq_dict: dict):
        """Возвращает два маппинга: прямой и обратный, для шифрования текстов случайной перестановкой"""
        alfabet = list(freq_dict.keys())
        alfabet_copy = alfabet.copy()
        random.shuffle(alfabet_copy)
        mapping = dict(zip(alfabet, alfabet_copy))
        return mapping

    @staticmethod
    def _map_text(text: str, mapping: dict):
        """Заменяет символы в тексте согласно мапингу"""
        f = lambda x: mapping.get(x, "%")
        result = "".join(list(map(f, text)))
        return result

    @staticmethod
    def _create_mapping_by_freq_dicts(freq_dict_1: dict, freq_dict_2: dict):
        """Создает маппинг для расшифровки текста на основе частотных dicts.
        freq_dict_1: частотный dict для корпуса текста на известном языке (отсортирован по убыванию частоты)
        freq_dict_2: частотный dict для неизвестного текста (отсортирован по убыванию частоты)
        """
        alfabet_1 = list(freq_dict_1.keys())
        alfabet_2 = list(freq_dict_2.keys())
        min_len = min(len(alfabet_1), len(alfabet_2))
        alfabet_1 = alfabet_1[:min_len]
        alfabet_2 = alfabet_2[:min_len]
        return dict(zip(alfabet_2, alfabet_1))


## Read and clean data

In [3]:
def read_and_clean_text(path: str):
    with open(path) as f:
        lines = f.readlines()
    text = " ".join(lines)
    clean_text = re.sub("\W+", " ", text).lower()
    return clean_text

In [4]:
text = read_and_clean_text("../corpora/WarAndPeace.txt")
text_en = read_and_clean_text("../corpora/WarAndPeaceEng.txt")
text_anna = read_and_clean_text("../corpora/AnnaKarenina.txt")

In [5]:
# Тестовые куски из Анны Карениной

test_sample = text_anna[650_106:651_060]
test_sample

'вронский понял тоже как мог быть силен серпуховской своею несомненною способностью обдумывать понимать вещи своим умом и даром слова так редко встречающимся в той среде в которой он жил и как ни совестно это было ему ему было завидно все таки мне недостает для этого одной главной вещи отвечал он недостает желания власти это было но прошло извини меня это неправда улыбаясь сказал серпуховской нет правда правда теперь чтоб быть искренним прибавил вронский да правда теперь это другое дело но это теперь будет не всегда может быть отвечал вронский ты говоришь может быть продолжал серпуховской как будто угадав его мысли а я тебе говорю наверное и для этого я хотел тебя видеть ты поступил так как должно было это я понимаю но персеверировать ты не должен я только прошу у тебя carte blanche 128 я не покровительствую тебе хотя отчего же мне и не покровительствовать тебе ты столько раз мне покровительствовал надеюсь что наша дружба стоит выше этого да'

In [6]:
test_sample2 = text_anna[450_076:451_058]
test_sample2

'она не слышала половины его слов она испытывала страх к нему и думала о том правда ли то что вронский не убился о нем ли говорили что он цел а лошадь сломала спину она только притворно насмешливо улыбнулась когда он кончил и ничего не отвечала потому что не слыхала того что он говорил алексей александрович начал говорить смело но когда он ясно понял то о чем он говорит страх который она испытывала сообщился ему он увидел эту улыбку и странное заблуждение нашло на него она улыбается над моими подозрениями да она скажет сейчас то что говорила мне тот раз что нет оснований моим подозрениям что это смешно теперь когда над ним висело открытие всего он ничего так не желал как того чтобы она так же как прежде насмешливо ответила ему что его подозрения смешны и не имеют основания так страшно было то что он знал что теперь он был готов поверить всему но выражение лица ее испуганного и мрачного теперь не обещало даже обмана может быть я ошибаюсь сказал он в таком случае я прошу'

## Baseline frequency method

Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [11]:
# Вспомогательный класс для шифровки/расшифровки текстов частотным методом
text_encryptor = TextEncryptor()

# "Обучаем" на частотах символов русской версии Войны и Мир
text_encryptor.fit(text)

# Шифруем тестовые куски текста из Анны Карениной случайной перестановкой
mapped_test_sample = text_encryptor.encrypt_random(test_sample)
mapped_test_sample2 = text_encryptor.encrypt_random(test_sample2)
print(f"Тестовые зашифрованные перестановкой куски:\n{mapped_test_sample[:80]},\n{mapped_test_sample2[:80]}")

# Расшифруем тестовые куски частотным методом
decrypted_test_sample = text_encryptor.decrypt_frequency_method(mapped_test_sample)
decrypted_test_sample2 = text_encryptor.decrypt_frequency_method(mapped_test_sample2)

Тестовые зашифрованные перестановкой куски:
7о5îéтбjqв5îgüqй5nцqтâтqd5êqwшй4qéбüцîqéцовet57éт5jqé75цэqîцé5dîцîî5эqéв5é5wî5éй,
5îâqîцqéüшvâüâqв5ü57бîшqцê5qéü57q5îâqбéвшйш7âüâqéйоâtqтqîцdeqбqûedâüâq5qй5dqвоâ7


In [8]:
decrypted_test_sample

'нвотсдлб мотыр еоча дид яоз уьеп слрат савмгхонсдоб сноаж тасоятаттож смосоутосепж оукгяьниеп мотляиеп нацл сноля гяоя л кивоя срони еид вакдо нсевашижцлясы н еоб свака н доеовоб от члр л дид тл сонасето йео уьро аяг аяг уьро юинлкто нса еидл ята такосеиае кры йеозо октоб зринтоб нацл оенашир от такосеиае чаритлы нрисел йео уьро то мвоeро люнлтл яаты йео тамвинки грьуиысп сдиюир савмгхонсдоб тае мвинки мвинки еамавп шеоу уьеп лсдваттля мвлуинлр нвотсдлб ки мвинки еамавп йео квгзоа каро то йео еамавп угкае та нсазки яочае уьеп оенашир нвотсдлб еь зоновлeп яочае уьеп мвокорчир савмгхонсдоб дид угкео гзикин азо яьсрл и ы еауа зоновж тинавтоа л кры йеозо ы хоеар еауы нлкаеп еь мосегмлр еид дид корчто уьро йео ы мотляиж то мавсанавлвониеп еь та корчат ы еорпдо мвоeг г еауы nsari uostnэi щlф ы та модвонлеарпсенгж еауа хоеы оешазо ча ята л та модвонлеарпсенониеп еауа еь сеорпдо вию ята модвонлеарпсенонир тикажсп шео тиeи квгчуи сеоле ньeа йеозо ки'

In [9]:
decrypted_test_sample2

'оеа еи лтябата готоксея иуо лток оеа слгяняката лнвах д еирп с ьпрата о нор гвакьа тс но мно квоелдсш еи пыстлз о еир тс уоковстс мно ое eит а тобаьч лтората лгсеп оеа нотчдо гвснковео еалрибтско птяыепталч доуьа ое доемст с есмиуо еи онкимата гонорп мно еи лтяхата ноуо мно ое уоковст атидлиш атидлаеьвоксм еамат уоковснч лрито ео доуьа ое злео гоезт но о мир ое уоковсн лнвах доновяш оеа слгяняката лооыюстлз ирп ое пксьит цнп птяыдп с лнваееои жаытпйьиеси еабто еа еиуо оеа птяыаинлз еаь росрс гоьожвиесзрс ьа оеа лдайин лишмал но мно уоковста реи нон важ мно еин олеокаесш роср гоьожвиесзр мно цно лрибео нигивч доуьа еаь еср кслито ондвянси клиуо ое есмиуо над еи йитат дад ноуо мноыя оеа над йи дад гвийьи еалрибтско онкинста ирп мно иуо гоьожвиесз лрибея с еи сриnн олеокаесз над лнвабео ыято но мно ое жеат мно нигивч ое ыят уонок гокивснч клирп ео кявайиеси тсeа ии слгпуаееоуо с рвамеоуо нигивч еи оыиюато ьайи оыраеа ройин ыянч з обсыаnлч лдажат ое к надор лтпмаи з гвобп'

* Расшифровка частотным методом с использованием посимвольной частоты получилась не очень.

## Bigram frequency method

Сделаем следующий логический шаг для улучшения:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.

In [15]:
import nltk
from nltk import bigrams
string = "I really like python, it's pretty awesome."
string_bigrams = bigrams(string)
[b[0]+b[1] for b in string_bigrams]

['I ',
 ' r',
 're',
 'ea',
 'al',
 'll',
 'ly',
 'y ',
 ' l',
 'li',
 'ik',
 'ke',
 'e ',
 ' p',
 'py',
 'yt',
 'th',
 'ho',
 'on',
 'n,',
 ', ',
 ' i',
 'it',
 "t'",
 "'s",
 's ',
 ' p',
 'pr',
 're',
 'et',
 'tt',
 'ty',
 'y ',
 ' a',
 'aw',
 'we',
 'es',
 'so',
 'om',
 'me',
 'e.']